# DES script for simulating hospital operations

In [420]:
import simpy
import networkx as nx
import numpy as np
import random
import pickle
import json

## Load the graph

In [421]:
with open('C:/Users/SID-DRW/OneDrive - Delft University of Technology/Documents/GitHub/HDES/Data/edited_graph.pkl', 'rb') as f:
    G = pickle.load(f)

## Load the shortest patient paths

In [422]:
with open('C:/Users/SID-DRW/OneDrive - Delft University of Technology/Documents/GitHub/HDES/Data/all_real_medical_path_1.pkl', 'rb') as f:
    all_real_medical_path_1 = pickle.load(f)

with open('C:/Users/SID-DRW/OneDrive - Delft University of Technology/Documents/GitHub/HDES/Data/all_real_medical_path_2.pkl', 'rb') as f:
    all_real_medical_path_2 = pickle.load(f)

with open('C:/Users/SID-DRW/OneDrive - Delft University of Technology/Documents/GitHub/HDES/Data/all_real_medical_path_3.pkl', 'rb') as f:
    all_real_medical_path_3 = pickle.load(f)

with open('C:/Users/SID-DRW/OneDrive - Delft University of Technology/Documents/GitHub/HDES/Data/all_real_pediatric_path_1.pkl', 'rb') as f:
    all_real_pediatric_path_1 = pickle.load(f)

with open('C:/Users/SID-DRW/OneDrive - Delft University of Technology/Documents/GitHub/HDES/Data/all_real_pediatric_path_2.pkl', 'rb') as f:
    all_real_pediatric_path_2 = pickle.load(f)

with open('C:/Users/SID-DRW/OneDrive - Delft University of Technology/Documents/GitHub/HDES/Data/all_real_pediatric_path_3.pkl', 'rb') as f:
    all_real_pediatric_path_3 = pickle.load(f)

In [423]:
print(all_real_medical_path_1)
print(all_real_medical_path_2)
print(all_real_medical_path_3)
print(all_real_pediatric_path_1)
print(all_real_pediatric_path_2)
print(all_real_pediatric_path_3)

[[44, 333, 7, 335, 0, 1, 152, 1, 302, 74, 301, 104, 301, 74, 302, 1, 0, 49, 266, 89], [44, 333, 7, 335, 0, 1, 152, 1, 302, 74, 301, 104, 11, 313, 207, 313, 11, 104, 301, 74, 302, 1, 0, 49, 266, 89], [44, 333, 7, 335, 0, 1, 152, 1, 302, 74, 301, 104, 11, 307, 208, 307, 11, 104, 301, 74, 302, 1, 0, 49, 266, 89], [44, 333, 7, 335, 0, 1, 152, 1, 302, 74, 301, 104, 11, 312, 170, 312, 11, 104, 301, 74, 302, 1, 0, 49, 266, 89], [44, 333, 7, 335, 0, 1, 152, 1, 302, 74, 301, 104, 11, 309, 71, 309, 11, 104, 301, 74, 302, 1, 0, 49, 266, 89], [44, 333, 7, 335, 0, 1, 152, 1, 302, 74, 301, 104, 11, 308, 138, 308, 11, 104, 301, 74, 302, 1, 0, 49, 266, 89], [44, 333, 7, 335, 0, 1, 152, 1, 302, 74, 301, 104, 11, 306, 85, 306, 11, 104, 301, 74, 302, 1, 0, 49, 266, 89]]
[[44, 333, 7, 335, 0, 1, 152, 1, 302, 74, 301, 104, 301, 74, 302, 1, 0, 338, 76, 336, 43, 336, 76, 338, 0, 49, 266, 89], [44, 333, 7, 335, 0, 1, 152, 1, 302, 74, 301, 104, 11, 313, 207, 313, 11, 104, 301, 74, 302, 1, 0, 338, 76, 336, 43, 

In [424]:
print(all_real_medical_path_1[0])
print(all_real_medical_path_1[1])
print(all_real_medical_path_1[2])
print(len(all_real_medical_path_1))
print(len(all_real_medical_path_2))
print(len(all_real_medical_path_3))
print(len(all_real_pediatric_path_1))
print(len(all_real_pediatric_path_2))
print(len(all_real_pediatric_path_3))

[44, 333, 7, 335, 0, 1, 152, 1, 302, 74, 301, 104, 301, 74, 302, 1, 0, 49, 266, 89]
[44, 333, 7, 335, 0, 1, 152, 1, 302, 74, 301, 104, 11, 313, 207, 313, 11, 104, 301, 74, 302, 1, 0, 49, 266, 89]
[44, 333, 7, 335, 0, 1, 152, 1, 302, 74, 301, 104, 11, 307, 208, 307, 11, 104, 301, 74, 302, 1, 0, 49, 266, 89]
7
7
7
5
5
5


# DES starts

### Initialize SimPy environment

In [425]:
env = simpy.Environment()

### Calculate each node's capacity (how many patients can it serve at one time)

#### - waiting area and corridors' capacities are 1m² per person, all other nodes' capacites are 1.

In [426]:

def get_capacity(G):

    # Extract nodes with 'name' attribute containing 'CORRIDOR' or 'WAITING'
    extracted_nodes = [node for node in G.nodes(data=True) if 'CORRIDOR' in node[1]['name'] or 'WAITING' in node[1]['name']]

    # Add 'capacity' attribute to each extracted node
    for node in extracted_nodes:
        node_index, node_data = node
        node_data['capacity'] = int(float(node_data['area']))  # Convert to float first, then to int

    # Create the capacities dictionary
    capacities = {}

    for node in G.nodes(data=True):
        node_index, node_data = node
        # If 'capacity' attribute exists, use it, otherwise assign 1
        capacities[node_index] = node_data.get('capacity', 1)

    return capacities

capacities = get_capacity(G)
print(capacities)

{0: 127, 1: 178, 2: 1, 3: 50, 4: 1, 5: 1, 6: 25, 7: 27, 8: 1, 9: 40, 10: 1, 11: 29, 12: 1, 13: 1, 14: 41, 15: 24, 16: 20, 17: 1, 18: 39, 19: 10, 20: 1, 21: 29, 22: 27, 23: 1, 24: 31, 25: 16, 26: 19, 27: 28, 28: 15, 29: 1, 30: 13, 31: 12, 32: 1, 33: 13, 34: 1, 35: 14, 36: 17, 37: 1, 38: 10, 39: 1, 40: 1, 41: 1, 42: 1, 43: 1, 44: 1, 45: 1, 46: 1, 47: 11, 48: 1, 49: 19, 50: 1, 51: 13, 52: 1, 53: 7, 54: 20, 55: 1, 56: 1, 57: 1, 58: 1, 59: 1, 60: 1, 61: 1, 62: 1, 63: 8, 64: 1, 65: 1, 66: 1, 67: 1, 68: 1, 69: 8, 70: 1, 71: 1, 72: 1, 73: 1, 74: 5, 75: 5, 76: 5, 77: 1, 78: 1, 79: 1, 80: 1, 81: 9, 82: 1, 83: 1, 84: 1, 85: 1, 86: 1, 87: 1, 88: 1, 89: 1, 90: 7, 91: 1, 92: 1, 93: 1, 94: 1, 95: 1, 96: 1, 97: 1, 98: 1, 99: 12, 100: 1, 101: 1, 102: 1, 103: 1, 104: 1, 105: 1, 106: 1, 107: 1, 108: 1, 109: 1, 110: 1, 111: 1, 112: 1, 113: 1, 114: 1, 115: 1, 116: 1, 117: 1, 118: 1, 119: 11, 120: 1, 121: 1, 122: 1, 123: 1, 124: 1, 125: 1, 126: 1, 127: 1, 128: 1, 129: 11, 130: 7, 131: 1, 132: 1, 133: 1, 134

In [427]:
# according to capacity, Create resources at nodes
resources = {node: simpy.Resource(env, capacity=capacities[node]) for node in G.nodes}

### Data tracking

In [428]:
waiting_times = {node: [] for node in G.nodes}
walking_distances = []  # Track walking distance for each patient
patient_waiting_times = []  # Track total waiting time for each patient
crowding = {node: [] for node in G.nodes} #crowdedness
patients_in_node = {node: 0 for node in G.nodes}
patient_travel_paths = {}  # Dictionary to store each patient's journey

# Track occupancy at each node over time
occupancy = {node: [] for node in G.nodes}

### Function to calculate distance between two nodes

In [429]:
def calculate_distance(node1, node2):
    pos1 = np.array(G.nodes[node1]['coords'])
    pos2 = np.array(G.nodes[node2]['coords'])
    return np.linalg.norm(pos1 - pos2)

### Function for creating Patient process (patient goes through patient path)

In [430]:
def patient(env, patient_id, path):
    total_walking_distance = 0
    total_waiting_time = 0
    journey = []  

    for i, node in enumerate(path):
        journey.append(node)  

        # Calculate walking distance 
        if i > 0:
            dist = calculate_distance(path[i-1], node)
            total_walking_distance += dist

        # calculate waiting time
        if capacities[node] > 0:  
            arrival_time = env.now
            with resources[node].request() as req:
                patients_in_node[node] += 1  # Update crowding
                yield req
                
                waiting_time = env.now - arrival_time
                total_waiting_time += waiting_time
                waiting_times[node].append(waiting_time)

        # simulate treatment (random service time: 15)
                yield env.timeout(np.random.exponential(15))  

        # calculate crowdedness (people density)
                occupancy[node].append(patients_in_node[node])

        crowding[node].append(patients_in_node[node] / float(G.nodes[node]['area']) if G.nodes[node]['area'] != 'NA' else 0)
        patients_in_node[node] -= 1  # Patient leaves the node

    # Record the walking distance, waiting time, and travel path for this patient
    walking_distances.append(total_walking_distance)
    patient_waiting_times.append(total_waiting_time)
    patient_travel_paths[f'patient_{patient_id}'] = journey  


### Function for choosing any available interaction room

Since there are multiple interaction rooms for patient to choose, 

this function ensures that patient can choose any available room, so it saves waiting time.

In [431]:
def get_available_medical_paths(resources,all_meidcal_path):
    available_paths = []
    if resources[104].count < resources[104].capacity:  
        available_paths.append(all_meidcal_path[0])
    if resources[207].count < resources[207].capacity:  
        available_paths.append(all_meidcal_path[1])
    if resources[208].count < resources[208].capacity:  
        available_paths.append(all_meidcal_path[2])
    if resources[170].count < resources[170].capacity:  
        available_paths.append(all_meidcal_path[3])
    if resources[71].count < resources[71].capacity:  
        available_paths.append(all_meidcal_path[4])
    if resources[138].count < resources[138].capacity:  
        available_paths.append(all_meidcal_path[5])
    if resources[85].count < resources[85].capacity:  
        available_paths.append(all_meidcal_path[6])
    return available_paths

def get_available_pediatric_paths(resources,all_pediatric_path):
    available_paths = []
    if resources[211].count < resources[211].capacity:  
        available_paths.append(all_pediatric_path[0])
    if resources[210].count < resources[210].capacity:  
        available_paths.append(all_pediatric_path[1])
    if resources[139].count < resources[139].capacity:  
        available_paths.append(all_pediatric_path[2])
    if resources[209].count < resources[209].capacity:  
        available_paths.append(all_pediatric_path[3])
    if resources[86].count < resources[86].capacity:  
        available_paths.append(all_pediatric_path[4])
    return available_paths

### Function for generating patients

In [432]:
def generate_patients(env, num_patients):
    for i in range(num_patients):
        # patient choose a path based on probabilities
        path_type_choice = random.choices(['medical_1', 'medical_2','medical_3',
                                           'pediatric_1','pediatric_2','pediatric_3'], 
                                           weights=[0.49, 0.05,0.01,0.39,0.05,0.01])[0]
        # if patient chooses medical path 1, the patient can choose any available interaction rooms along this path
        if path_type_choice == 'medical_1':
            available_paths = get_available_medical_paths(resources,all_real_medical_path_1) 
            
            
            if available_paths:
                chosen_path = random.choice(available_paths)
                env.process(patient(env, i, chosen_path))
            else:
                # If all ERs are occupied, the patient must wait 
                yield env.timeout(1)  # Small wait time before retrying

        # if patient chooses medical path 2, the patient can choose any available interaction rooms along this path
        if path_type_choice == 'medical_2':
            available_paths = get_available_medical_paths(resources,all_real_medical_path_2)
            
            if available_paths:
                chosen_path = random.choice(available_paths)
                env.process(patient(env, i, chosen_path))
            else:
                
                yield env.timeout(1)  

        # if patient chooses medical path 3, the patient can choose any available interaction rooms along this path
        if path_type_choice == 'medical_3':
            available_paths = get_available_medical_paths(resources,all_real_medical_path_3)
            
            if available_paths:
                chosen_path = random.choice(available_paths)
                env.process(patient(env, i, chosen_path))
            else:
                
                yield env.timeout(1)  

        # if patient chooses pediatric path 1, the patient can choose any available interaction rooms along this path
        if path_type_choice == 'pediatric_1':
            available_paths = get_available_pediatric_paths(resources,all_real_pediatric_path_1)
            
            if available_paths:
                chosen_path = random.choice(available_paths)
                env.process(patient(env, i, chosen_path))
            else:
                
                yield env.timeout(1)  
        
        # if patient chooses pediatric path 2, the patient can choose any available interaction rooms along this path
        if path_type_choice == 'pediatric_2':
            available_paths = get_available_pediatric_paths(resources,all_real_pediatric_path_2)
            
            if available_paths:
                chosen_path = random.choice(available_paths)
                env.process(patient(env, i, chosen_path))
            else:
                 
                yield env.timeout(1)  

        # if patient chooses pediatric path 3, the patient can choose any available interaction rooms along this path
        if path_type_choice == 'pediatric_3':
            available_paths = get_available_pediatric_paths(resources,all_real_pediatric_path_3)
            
            if available_paths:
                chosen_path = random.choice(available_paths)
                env.process(patient(env, i, chosen_path))
            else:
                
                yield env.timeout(1)  

        yield env.timeout(np.random.exponential(10))  # Random patient arrival time


### Run the simulation & collect patient travel paths

In [433]:
env.process(generate_patients(env, 50))  # Simulate the hospital with 20 patients
env.run(until=720)  # Run the simulation for 1440 minutes

# print all patient travel paths
def extract_patient_number(key):
    return int(key.split('_')[1])

for key in sorted(patient_travel_paths.keys(), key=extract_patient_number):
    print(f"{key}, {patient_travel_paths[key]}")

patient_0, [44, 333, 7, 335, 0, 1, 324, 119, 471, 21, 470, 107, 470, 21, 471, 119, 471, 21, 472, 211, 472, 21, 471, 119, 324, 1, 0, 338, 76, 336, 43, 336, 76, 338, 0, 49, 266, 89]
patient_1, [44, 333, 7, 335, 0, 1, 152, 1, 302, 74, 301, 104, 11, 312, 170, 312, 11, 104, 301, 74, 302, 1, 0, 49, 266, 89]
patient_2, [44, 333, 7, 335, 0, 1, 152, 1, 302, 74, 301, 104, 11, 313, 207, 313, 11, 104, 301, 74, 302, 1, 0, 49, 266, 89]
patient_3, [44, 333, 7, 335, 0, 1, 152, 1, 302, 74, 301, 104, 11, 312, 170, 312, 11, 104, 301, 74, 302, 1, 0, 49, 266, 89]
patient_6, [44, 333, 7, 335, 0, 1, 152, 1, 302, 74, 301, 104, 11, 313, 207, 313, 11, 104, 301, 74, 302, 1, 0, 49, 266, 89]
patient_8, [44, 333, 7, 335, 0, 1, 152, 1, 302, 74, 301, 104, 11, 308, 138, 308, 11, 104, 301, 74, 302, 1, 0, 49, 266, 89]
patient_9, [44, 333, 7, 335, 0, 1, 152, 1, 302, 74, 301, 104, 301, 74, 302, 1, 0, 49, 266, 89]
patient_18, [44, 333, 7, 335, 0, 1, 152, 1, 302, 74, 301, 104, 301, 74, 302, 1, 0, 49, 266, 89]


# Assessing Difficulty in wayfinding 
### - by calculating extra walking distance

### load the random patient paths

In [434]:
with open('C:/Users/SID-DRW/OneDrive - Delft University of Technology/Documents/GitHub/HDES/Data/all_random_medical_path_1.pkl', 'rb') as f:
    all_real_medical_random_path_1 = pickle.load(f)

with open('C:/Users/SID-DRW/OneDrive - Delft University of Technology/Documents/GitHub/HDES/Data/all_random_medical_path_2.pkl', 'rb') as f:
    all_real_medical_random_path_2 = pickle.load(f)

with open('C:/Users/SID-DRW/OneDrive - Delft University of Technology/Documents/GitHub/HDES/Data/all_random_medical_path_3.pkl', 'rb') as f:
    all_real_medical_random_path_3 = pickle.load(f)

with open('C:/Users/SID-DRW/OneDrive - Delft University of Technology/Documents/GitHub/HDES/Data/all_random_pediatric_path_1.pkl', 'rb') as f:
    all_real_pediatric_random_path_1 = pickle.load(f)

with open('C:/Users/SID-DRW/OneDrive - Delft University of Technology/Documents/GitHub/HDES/Data/all_random_pediatric_path_2.pkl', 'rb') as f:
    all_real_pediatric_random_path_2 = pickle.load(f)

with open('C:/Users/SID-DRW/OneDrive - Delft University of Technology/Documents/GitHub/HDES/Data/all_random_pediatric_path_3.pkl', 'rb') as f:
    all_real_pediatric_random_path_3 = pickle.load(f)

### Calculate distance of patient path

In [435]:
# Function for calulating total distance of the path

import math

def calculate_path_distance(path, G):
    total_distance = 0.0

    # Iterate through consecutive nodes in the path
    for i in range(len(path) - 1):
        node1 = path[i]
        node2 = path[i + 1]
        
        # Get the 3D coordinates of both nodes
        x1, y1, z1 = G.nodes[node1]['coords']
        x2, y2, z2 = G.nodes[node2]['coords']
        
        # Calculate the Euclidean distance between node1 and node2
        distance = math.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2 + (z2 - z1) ** 2)
        
        # Add the distance to the total distance
        total_distance += distance

    return total_distance


In [436]:
# Function for calulating average distance of multiple paths

def get_total_distance_allPath(all_path,G):
    distances = []
    for path in all_path:
        single_distance = calculate_path_distance(path,G)
        distances.append(single_distance)
    final_total_distance = sum(distances)
    average = final_total_distance / len(distances)

    return average

In [437]:
# Average distance of the random paths
all_distance_m1 = get_total_distance_allPath(all_real_medical_random_path_1,G)
print(all_distance_m1)

all_distance_m2 = get_total_distance_allPath(all_real_medical_random_path_2,G)
print(all_distance_m2)

all_distance_m3 = get_total_distance_allPath(all_real_medical_random_path_3,G)
print(all_distance_m3)

all_distance_p1 = get_total_distance_allPath(all_real_pediatric_random_path_1,G)
print(all_distance_p1)  

all_distance_p2 = get_total_distance_allPath(all_real_pediatric_random_path_2,G)
print(all_distance_p2) 

all_distance_p3 = get_total_distance_allPath(all_real_pediatric_random_path_1,G)
print(all_distance_p3) 

18069.63004523108
19977.849241349883
15597.284611218853
32775.01567488796
26466.013116426
32775.01567488796


In [438]:
# average distance of shortest paths

all_shortest_distance_m1 = get_total_distance_allPath(all_real_medical_path_1,G)
print(all_shortest_distance_m1)

all_shortest_distance_m2 = get_total_distance_allPath(all_real_medical_path_2,G)
print(all_shortest_distance_m2)

all_shortest_distance_m3 = get_total_distance_allPath(all_real_medical_path_3,G)
print(all_shortest_distance_m3)

all_shortest_distance_p1 = get_total_distance_allPath(all_real_pediatric_path_1,G)
print(all_shortest_distance_p1)

all_shortest_distance_p2 = get_total_distance_allPath(all_real_pediatric_path_2,G)
print(all_shortest_distance_p2)

all_shortest_distance_p3 = get_total_distance_allPath(all_real_pediatric_path_3,G)
print(all_shortest_distance_p3)

155.7344233731371
179.05542790783412
191.19252675596076
149.22439960211142
172.54540413680843
184.68250298493507


### Extra walking distance

In [439]:
def get_extra_walking_distance(random_d1,random_d2,random_d3,
                               random_p1,random_p2,random_p3,
                               shortest_d1,shortest_d2,shortest_d3,
                               shortest_p1,shortest_p2,shortest_p3):
    random_average_distance = 0.49*random_d1+0.05*random_d2+0.01*random_d3+0.39*random_p1+0.05*random_p2+0.01*random_p3
    shortest_average_distance = 0.49*shortest_d1+0.05*shortest_d2+0.01*shortest_d3+0.39*shortest_p1+0.05*shortest_p2+0.01*shortest_p3

    extra_distance = random_average_distance - shortest_average_distance

    return extra_distance

In [440]:
extra_distance = get_extra_walking_distance(all_distance_m1,all_distance_m2,all_distance_m3,
                                            all_distance_p1,all_distance_p2,all_distance_p3,
                                            all_shortest_distance_m1,all_shortest_distance_m2,all_shortest_distance_m3,
                                            all_shortest_distance_p1,all_shortest_distance_p2,all_shortest_distance_p3)

print(extra_distance)

24286.444780922095


# Collect simulation results

In [441]:
# After simulation, analyze the collected data
average_walking_distance = np.mean(walking_distances)
average_patient_waiting_time = np.mean(patient_waiting_times)

print("Average waiting times at each node:", {node: np.mean(waiting_times[node]) for node in waiting_times if waiting_times[node]})
print("Average patient walking distance:", average_walking_distance)
print("Average patient waiting time:", average_patient_waiting_time)
print("Average crowding at each node:", {node: np.mean(crowding[node]) for node in crowding if crowding[node]})
print("Average extra walking distance:", extra_distance)


Average waiting times at each node: {0: 0.0, 1: 0.0, 7: 0.0, 11: 0.0, 21: 0.0, 43: 0.0, 44: 139.63973529707567, 49: 0.0, 71: 0.0, 74: 0.0, 76: 0.0, 85: 0.0, 86: 5.426761160782689, 89: 3.7270416203553935, 104: 33.880683234230816, 107: 0.009038610272871969, 119: 0.0, 138: 0.0, 152: 5.105542544079613, 170: 0.0, 207: 0.0, 208: 0.0, 209: 0.0, 210: 0.0, 211: 0.0, 266: 2.7493862804510343, 301: 15.930690677481417, 302: 18.566815621027413, 306: 0.0, 307: 12.814531101547203, 308: 3.689930747353941, 309: 0.0, 312: 0.0, 313: 0.0, 324: 7.890749410093229, 327: 0.0, 328: 28.58584900553251, 329: 0.0, 333: 33.61934125598593, 335: 10.141025025553002, 336: 0.0, 338: 0.0, 470: 12.685337078625114, 471: 22.057964831579497, 472: 0.0}
Average patient walking distance: 153.07871708966235
Average patient waiting time: 157.73198098331648
Average crowding at each node: {0: 0.016077807148996766, 1: 0.013448917833505204, 7: 0.07410648613803435, 11: 0.0536476696111237, 21: 0.06705796160779141, 43: 0.1135294601683251